<a href="https://colab.research.google.com/github/annmarieakuba/AnnmarieMensah._SportsPrediction/blob/main/AnnmarieMensah__SportsPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer as si
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor, AdaBoostRegressor
from sklearn.model_selection import RandomizedSearchCV, KFold, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from scipy.stats import norm

Import datasets for training the model and testing the model with new data

In [ ]:
legacy_dataset = pd.read_csv('/content/drive/MyDrive/legacymaleplayers.csv')
players_dataset = pd.read_csv('/content/drive/MyDrive/players_22-1-3.csv')

<ipython-input-357-ce97a4b597da>:1: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  legacy_dataset = pd.read_csv('/content/drive/MyDrive/legacymaleplayers.csv')
<ipython-input-357-ce97a4b597da>:2: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  players_dataset = pd.read_csv('/content/drive/MyDrive/players_22-1-3.csv')


In [ ]:
players_dataset.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png


In [ ]:
players_dataset.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 110 columns):
 #    Column                       Non-Null Count  Dtype  
---   ------                       --------------  -----  
 0    sofifa_id                    19239 non-null  int64  
 1    player_url                   19239 non-null  object 
 2    short_name                   19239 non-null  object 
 3    long_name                    19239 non-null  object 
 4    player_positions             19239 non-null  object 
 5    overall                      19239 non-null  int64  
 6    potential                    19239 non-null  int64  
 7    value_eur                    19165 non-null  float64
 8    wage_eur                     19178 non-null  float64
 9    age                          19239 non-null  int64  
 10   dob                          19239 non-null  object 
 11   height_cm                    19239 non-null  int64  
 12   weight_kg                    19239 non-null  int64  
 13  

#Preprocessing the data

Remove the clearly useless variables according to the description of the attributes in the dataset.

In [ ]:
# take out url and id values because they are categorised as useless
useless = ['player_face_url', 'club_logo_url', 'club_flag_url', 'nation_logo_url', 'nation_flag_url', 'sofifa_id', 'player_url','gk', 'rb', 'rcb', 'cb', 'lcb', 'lb', 'rwb', 'rdm', 'cdm', 'ldm', 'lwb', 'rm', 'rcm', 'cm', 'lcm', 'lm', 'ram', 'cam', 'lam', 'rw', 'rf', 'cf', 'lf', 'lw', 'rs', 'st', 'ls']
players_dataset.drop(columns=useless, inplace=True)
players_dataset

,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
0,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,1987-06-24,170,...,96,20,35,24,6,11,15,14,8,NaN
1,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,1988-08-21,185,...,88,35,42,19,15,6,12,8,10,NaN
2,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,1985-02-05,187,...,95,24,32,24,7,11,15,14,11,NaN
3,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,1992-02-05,175,...,93,35,32,29,9,9,15,15,11,NaN
4,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,1991-06-28,181,...,89,68,65,53,15,13,5,10,13,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,Song Defu,宋德福,CDM,47,52,70000.0,1000.0,22,1999-01-02,180,...,37,38,43,48,6,10,5,15,13,NaN
19235,C. Porter,Caoimhin Porter,CM,47,59,110000.0,500.0,19,2002-03-18,175,...,47,37,44,47,11,12,6,8,10,NaN
19236,N. Logue,Nathan Logue-Cunningham,CM,47,55,100000.0,500.0,21,2000-02-07,178,...,36,38,44,48,8,6,7,10,6,NaN
19237,L. Rudden,Luke Rudden,ST,47,60,110000.0,500.0,19,2002-04-23,173,...,47,10,14,11,7,10,7,14,15,NaN


Remove attributes with 30% or more missing values

In [ ]:
null_attributes = players_dataset.columns[players_dataset.count()/18944 < 0.7]
players_dataset.drop(columns=null_attributes, inplace=True)

In [ ]:
players_dataset.shape

(19239, 69)

In [ ]:
players_dataset

,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,...,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,1987-06-24,170,...,75,96,20,35,24,6,11,15,14,8
1,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,1988-08-21,185,...,90,88,35,42,19,15,6,12,8,10
2,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,1985-02-05,187,...,88,95,24,32,24,7,11,15,14,11
3,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,1992-02-05,175,...,93,93,35,32,29,9,9,15,15,11
4,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,1991-06-28,181,...,83,89,68,65,53,15,13,5,10,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,Song Defu,宋德福,CDM,47,52,70000.0,1000.0,22,1999-01-02,180,...,42,37,38,43,48,6,10,5,15,13
19235,C. Porter,Caoimhin Porter,CM,47,59,110000.0,500.0,19,2002-03-18,175,...,44,47,37,44,47,11,12,6,8,10
19236,N. Logue,Nathan Logue-Cunningham,CM,47,55,100000.0,500.0,21,2000-02-07,178,...,39,36,38,44,48,8,6,7,10,6
19237,L. Rudden,Luke Rudden,ST,47,60,110000.0,500.0,19,2002-04-23,173,...,49,47,10,14,11,7,10,7,14,15


Extract and Show all categorical attributes based on the data types

In [ ]:
categorical_df = players_dataset.select_dtypes(include=['object'])
categorical_df

,short_name,long_name,player_positions,dob,club_name,league_name,club_position,club_joined,nationality_name,preferred_foot,work_rate,body_type,real_face
0,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",1987-06-24,Paris Saint-Germain,French Ligue 1,RW,2021-08-10,Argentina,Left,Medium/Low,Unique,Yes
1,R. Lewandowski,Robert Lewandowski,ST,1988-08-21,FC Bayern München,German 1. Bundesliga,ST,2014-07-01,Poland,Right,High/Medium,Unique,Yes
2,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",1985-02-05,Manchester United,English Premier League,ST,2021-08-27,Portugal,Right,High/Low,Unique,Yes
3,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",1992-02-05,Paris Saint-Germain,French Ligue 1,LW,2017-08-03,Brazil,Right,High/Medium,Unique,Yes
4,K. De Bruyne,Kevin De Bruyne,"CM, CAM",1991-06-28,Manchester City,English Premier League,RCM,2015-08-30,Belgium,Right,High/High,Unique,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,Song Defu,宋德福,CDM,1999-01-02,Wuhan FC,Chinese Super League,SUB,2021-04-01,China PR,Right,Medium/Medium,Lean (170-185),No
19235,C. Porter,Caoimhin Porter,CM,2002-03-18,Derry City,Rep. Ireland Airtricity League,RES,2021-02-22,Republic of Ireland,Right,Medium/Medium,Lean (170-185),No
19236,N. Logue,Nathan Logue-Cunningham,CM,2000-02-07,Finn Harps,Rep. Ireland Airtricity League,RES,2021-06-18,Republic of Ireland,Right,Medium/Medium,Lean (170-185),No
19237,L. Rudden,Luke Rudden,ST,2002-04-23,Finn Harps,Rep. Ireland Airtricity League,RES,2021-06-18,Republic of Ireland,Right,Medium/Medium,Lean (170-185),No


Select and show only the attributes that are numerical attributes

In [ ]:
numerical_df = players_dataset.select_dtypes(exclude=['object'])
numerical_df

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,club_jersey_number,...,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,93,93,78000000.0,320000.0,34,170,72,73.0,1.0,30.0,...,75,96,20,35,24,6,11,15,14,8
1,92,92,119500000.0,270000.0,32,185,81,21.0,1.0,9.0,...,90,88,35,42,19,15,6,12,8,10
2,91,91,45000000.0,270000.0,36,187,83,11.0,1.0,7.0,...,88,95,24,32,24,7,11,15,14,11
3,91,91,129000000.0,270000.0,29,175,68,73.0,1.0,10.0,...,93,93,35,32,29,9,9,15,15,11
4,91,91,125500000.0,350000.0,30,181,70,10.0,1.0,17.0,...,83,89,68,65,53,15,13,5,10,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,47,52,70000.0,1000.0,22,180,64,112541.0,1.0,36.0,...,42,37,38,43,48,6,10,5,15,13
19235,47,59,110000.0,500.0,19,175,70,445.0,1.0,27.0,...,44,47,37,44,47,11,12,6,8,10
19236,47,55,100000.0,500.0,21,178,72,111131.0,1.0,31.0,...,39,36,38,44,48,8,6,7,10,6
19237,47,60,110000.0,500.0,19,173,66,111131.0,1.0,12.0,...,49,47,10,14,11,7,10,7,14,15


In [ ]:
# Show to check which attributes have null values in the categorical dataset
categorical_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   short_name        19239 non-null  object
 1   long_name         19239 non-null  object
 2   player_positions  19239 non-null  object
 3   dob               19239 non-null  object
 4   club_name         19178 non-null  object
 5   league_name       19178 non-null  object
 6   club_position     19178 non-null  object
 7   club_joined       18076 non-null  object
 8   nationality_name  19239 non-null  object
 9   preferred_foot    19239 non-null  object
 10  work_rate         19239 non-null  object
 11  body_type         19239 non-null  object
 12  real_face         19239 non-null  object
dtypes: object(13)
memory usage: 1.9+ MB


Select the attributes with null values based on the count of non-null rows and impute them using a forward fill

In [ ]:
# Select categorical attributes with null values
columns_to_fill = categorical_df.columns[categorical_df.count() < 18944]
columns_to_fill


Index(['club_joined'], dtype='object')

In [ ]:
# Forward fill attributes with na values
categorical_df[columns_to_fill] = categorical_df[columns_to_fill].fillna(method='ffill')

In [ ]:
# Check to see that there are no null values
categorical_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   short_name        19239 non-null  object
 1   long_name         19239 non-null  object
 2   player_positions  19239 non-null  object
 3   dob               19239 non-null  object
 4   club_name         19178 non-null  object
 5   league_name       19178 non-null  object
 6   club_position     19178 non-null  object
 7   club_joined       19239 non-null  object
 8   nationality_name  19239 non-null  object
 9   preferred_foot    19239 non-null  object
 10  work_rate         19239 non-null  object
 11  body_type         19239 non-null  object
 12  real_face         19239 non-null  object
dtypes: object(13)
memory usage: 1.9+ MB


Create an object to encode the categorical variables and fit_transform all the cells inside of the data frame that contains just categorical variables

In [ ]:
encoder = LabelEncoder()
categorical_df= categorical_df.apply(encoder.fit_transform)
categorical_df

,short_name,long_name,player_positions,dob,club_name,league_name,club_position,club_joined,nationality_name,preferred_foot,work_rate,body_type,real_face
0,10117,10421,604,719,470,19,25,1808,6,0,7,9,1
1,14468,14882,635,1016,241,21,27,210,123,1,2,9,1
2,3288,3379,658,266,412,17,27,1825,124,1,1,9,1
3,13097,13094,372,2143,470,19,14,652,20,1,2,9,1
4,9005,9692,168,1940,411,17,19,353,13,1,0,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,16139,18367,98,4570,688,6,28,1682,30,1,8,1,0
19235,2947,2673,167,5699,212,39,21,1645,126,1,8,1,0
19236,12809,12992,167,4958,291,39,21,1755,126,1,8,1,0
19237,10276,10917,635,5735,291,39,21,1755,126,1,8,1,0


Create object and impute the data frame containing numerical attributes to get rid of null values

In [ ]:
# Impute numerical attributes
imputer = si(strategy='median')

In [ ]:

# checking if null values exist in numerical attributes
numerical_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 56 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   overall                      19239 non-null  int64  
 1   potential                    19239 non-null  int64  
 2   value_eur                    19165 non-null  float64
 3   wage_eur                     19178 non-null  float64
 4   age                          19239 non-null  int64  
 5   height_cm                    19239 non-null  int64  
 6   weight_kg                    19239 non-null  int64  
 7   club_team_id                 19178 non-null  float64
 8   league_level                 19178 non-null  float64
 9   club_jersey_number           19178 non-null  float64
 10  club_contract_valid_until    19178 non-null  float64
 11  nationality_id               19239 non-null  int64  
 12  weak_foot                    19239 non-null  int64  
 13  skill_moves     

In [ ]:

imputed_numerical = imputer.fit_transform(numerical_df)
imputed_numerical_df = pd.DataFrame(imputed_numerical, columns=numerical_df.columns)
imputed_numerical_df

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,club_jersey_number,...,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,93.0,93.0,78000000.0,320000.0,34.0,170.0,72.0,73.0,1.0,30.0,...,75.0,96.0,20.0,35.0,24.0,6.0,11.0,15.0,14.0,8.0
1,92.0,92.0,119500000.0,270000.0,32.0,185.0,81.0,21.0,1.0,9.0,...,90.0,88.0,35.0,42.0,19.0,15.0,6.0,12.0,8.0,10.0
2,91.0,91.0,45000000.0,270000.0,36.0,187.0,83.0,11.0,1.0,7.0,...,88.0,95.0,24.0,32.0,24.0,7.0,11.0,15.0,14.0,11.0
3,91.0,91.0,129000000.0,270000.0,29.0,175.0,68.0,73.0,1.0,10.0,...,93.0,93.0,35.0,32.0,29.0,9.0,9.0,15.0,15.0,11.0
4,91.0,91.0,125500000.0,350000.0,30.0,181.0,70.0,10.0,1.0,17.0,...,83.0,89.0,68.0,65.0,53.0,15.0,13.0,5.0,10.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,47.0,52.0,70000.0,1000.0,22.0,180.0,64.0,112541.0,1.0,36.0,...,42.0,37.0,38.0,43.0,48.0,6.0,10.0,5.0,15.0,13.0
19235,47.0,59.0,110000.0,500.0,19.0,175.0,70.0,445.0,1.0,27.0,...,44.0,47.0,37.0,44.0,47.0,11.0,12.0,6.0,8.0,10.0
19236,47.0,55.0,100000.0,500.0,21.0,178.0,72.0,111131.0,1.0,31.0,...,39.0,36.0,38.0,44.0,48.0,8.0,6.0,7.0,10.0,6.0
19237,47.0,60.0,110000.0,500.0,19.0,173.0,66.0,111131.0,1.0,12.0,...,49.0,47.0,10.0,14.0,11.0,7.0,10.0,7.0,14.0,15.0


Combine both the dataframes of the the categorical and numerical attributes into a single data frame

In [ ]:

new_legacy_dataset = pd.concat([imputed_numerical_df, categorical_df], axis=1)
new_legacy_dataset

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,club_jersey_number,...,dob,club_name,league_name,club_position,club_joined,nationality_name,preferred_foot,work_rate,body_type,real_face
0,93.0,93.0,78000000.0,320000.0,34.0,170.0,72.0,73.0,1.0,30.0,...,719,470,19,25,1808,6,0,7,9,1
1,92.0,92.0,119500000.0,270000.0,32.0,185.0,81.0,21.0,1.0,9.0,...,1016,241,21,27,210,123,1,2,9,1
2,91.0,91.0,45000000.0,270000.0,36.0,187.0,83.0,11.0,1.0,7.0,...,266,412,17,27,1825,124,1,1,9,1
3,91.0,91.0,129000000.0,270000.0,29.0,175.0,68.0,73.0,1.0,10.0,...,2143,470,19,14,652,20,1,2,9,1
4,91.0,91.0,125500000.0,350000.0,30.0,181.0,70.0,10.0,1.0,17.0,...,1940,411,17,19,353,13,1,0,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,47.0,52.0,70000.0,1000.0,22.0,180.0,64.0,112541.0,1.0,36.0,...,4570,688,6,28,1682,30,1,8,1,0
19235,47.0,59.0,110000.0,500.0,19.0,175.0,70.0,445.0,1.0,27.0,...,5699,212,39,21,1645,126,1,8,1,0
19236,47.0,55.0,100000.0,500.0,21.0,178.0,72.0,111131.0,1.0,31.0,...,4958,291,39,21,1755,126,1,8,1,0
19237,47.0,60.0,110000.0,500.0,19.0,173.0,66.0,111131.0,1.0,12.0,...,5735,291,39,21,1755,126,1,8,1,0


In [ ]:
# Confirm that there are no null values
new_legacy_dataset.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 69 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   overall                      19239 non-null  float64
 1   potential                    19239 non-null  float64
 2   value_eur                    19239 non-null  float64
 3   wage_eur                     19239 non-null  float64
 4   age                          19239 non-null  float64
 5   height_cm                    19239 non-null  float64
 6   weight_kg                    19239 non-null  float64
 7   club_team_id                 19239 non-null  float64
 8   league_level                 19239 non-null  float64
 9   club_jersey_number           19239 non-null  float64
 10  club_contract_valid_until    19239 non-null  float64
 11  nationality_id               19239 non-null  float64
 12  weak_foot                    19239 non-null  float64
 13  skill_moves     

#Feature Engineering
**Select the needed attributes to train the regression model with**

Here, I picked attributes with a correlation of more than |0.5| to ensure that we get the attributes with a strong correlation.

In [ ]:
correlation_vals = new_legacy_dataset.corr()['overall'] # check correlation of other attributes with overall
corr_attributes = correlation_vals[abs(correlation_vals.values) >= 0.5]  # which attributes have a strong correlation with the overall rating
corr_attributes.index

Index(['overall', 'potential', 'value_eur', 'wage_eur', 'release_clause_eur',
       'passing', 'dribbling', 'attacking_short_passing', 'movement_reactions',
       'power_shot_power', 'mentality_vision', 'mentality_composure'],
      dtype='object')

**Create new dataframe of needed attributes**

In [ ]:
needed_att_legacy = new_legacy_dataset[corr_attributes.index]
needed_att_legacy

,overall,potential,value_eur,wage_eur,release_clause_eur,passing,dribbling,attacking_short_passing,movement_reactions,power_shot_power,mentality_vision,mentality_composure
0,93.0,93.0,78000000.0,320000.0,144300000.0,91.0,95.0,91.0,94.0,86.0,95.0,96.0
1,92.0,92.0,119500000.0,270000.0,197200000.0,79.0,86.0,85.0,93.0,90.0,81.0,88.0
2,91.0,91.0,45000000.0,270000.0,83300000.0,80.0,88.0,80.0,94.0,94.0,76.0,95.0
3,91.0,91.0,129000000.0,270000.0,238700000.0,86.0,94.0,86.0,89.0,80.0,90.0,93.0
4,91.0,91.0,125500000.0,350000.0,232200000.0,93.0,88.0,94.0,91.0,91.0,94.0,89.0
...,...,...,...,...,...,...,...,...,...,...,...,...
19234,47.0,52.0,70000.0,1000.0,114000.0,46.0,48.0,50.0,53.0,46.0,43.0,37.0
19235,47.0,59.0,110000.0,500.0,193000.0,50.0,46.0,51.0,49.0,49.0,49.0,47.0
19236,47.0,55.0,100000.0,500.0,175000.0,45.0,49.0,49.0,46.0,50.0,46.0,36.0
19237,47.0,60.0,110000.0,500.0,239000.0,36.0,48.0,38.0,48.0,48.0,40.0,47.0


**Scale dataset inputs only, by separating output attribute from the rest of the dataset**




This is to prevent overall score from being scaled. It was separated from the rest of the dataset and stored in a separate variable (overall_att) before going ahead to scale the input attributes

In [ ]:
overall_att = needed_att_legacy.overall

In [ ]:
model_legacy = overall_att.drop(columns='overall')
model_legacy

0        93.0
1        92.0
2        91.0
3        91.0
4        91.0
         ... 
19234    47.0
19235    47.0
19236    47.0
19237    47.0
19238    47.0
Name: overall, Length: 19239, dtype: float64

Create standard scalar object

In [ ]:
sc = StandardScaler()

In [ ]:
# Ensure model_legacy is a DataFrame
model_legacy = pd.DataFrame(model_legacy)

In [ ]:
# scale input values of data
scaled_data = sc.fit_transform(model_legacy)
ready_legacy_players= pd.DataFrame(scaled_data, columns=model_legacy.columns)
ready_legacy_players

,overall
0,3.957501
1,3.812154
2,3.666806
3,3.666806
4,3.666806
...,...
19234,-2.728494
19235,-2.728494
19236,-2.728494
19237,-2.728494


#Model development and testing

Using the test/train split and splitting the X(input attributes) and Y(Output attribute) to train and test the models

In [ ]:
X = ready_legacy_players
Y = overall_att

In [ ]:
# test/train split for cross-validation
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
Xtrain.shape

(15391, 1)

**Train, test, and measure accuracy of RandomForestRegressor model with training split of dataset and testing split of dataset**

In [ ]:
# Instantiate Regression model
rf_model = RandomForestRegressor(n_estimators=2000, max_depth=1000, n_jobs=-1, random_state=42)

In [ ]:
# train the regression model
rf_model.fit(Xtrain, Ytrain)

RandomForestRegressor(max_depth=1000, n_estimators=2000, n_jobs=-1,
                      random_state=42)

In [ ]:
# use model for prediction
model_prediction = rf_model.predict(Xtest)

In [ ]:
mean_abs_err_results = mean_absolute_error(Ytest, model_prediction)
mean_abs_err_results

0.00038591476091476447

**Train, test and measure accuracy of AdaBoostRegressor model with training split of dataset and testing split of dataset**

Create an AdaBoostRegressor with a base model as a decisiontreeregressor

In [ ]:
base_model = DecisionTreeRegressor(max_depth=1000)

In [ ]:

adaboost_regressor = AdaBoostRegressor(base_model, n_estimators=1000, random_state=42)

adaboost_regressor.fit(Xtrain, Ytrain)

AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=1000),
                  n_estimators=1000, random_state=42)

In [ ]:
ada_predict = adaboost_regressor.predict(Xtest)
mae = mean_absolute_error(Ytest, ada_predict)
mae

0.0002598752598752599

**Train, test, and measure accuracy of Gradient Boosting model with training split of dataset and testing split of dataset**

Create and train a GradientBoosting model (gb_model) Use the test data-set ( Xtest ) to make predictions and then check the accuracy using mean absolute error

In [ ]:

# Instantiate GradientBoosting model
gb_model = GradientBoostingRegressor(n_estimators=50, random_state=42)

In [ ]:
# Train the GradientBoosting model
gb_model.fit(Xtrain, Ytrain)

GradientBoostingRegressor(n_estimators=50, random_state=42)

In [ ]:
# use GradientBoosting model for prediction
gb_prediction = gb_model.predict(Xtest)

In [ ]:

gb_mean_abs_err = mean_absolute_error(Ytest, gb_prediction)
gb_mean_abs_err

0.03281081265922311

Train, test, and measure accuracy of XGboost model with training split of dataset and testing split of dataset

In [ ]:
# Create model that looks to minimize mean squared error as much as possible
xg_model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=50)

In [ ]:
xg_model.fit(Xtrain, Ytrain)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=50, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:

# predict with model
xg_prediction = xg_model.predict(Xtest)

In [ ]:
xg_mean_abs_err = mean_absolute_error(Ytest, xg_prediction)
xg_mean_abs_err

0.00027643667685019

Train, test and measure accuracy of VotingRegressor ensemble using all previously used models

In [ ]:

# Create model instance
soft_ensemble = VotingRegressor(estimators=[
    ('randomforest', rf_model),
    ('gradientboosing', gb_model),
    ('xgb', xg_model)
], weights=[0.5, 0.1, 0.4])

In [ ]:
soft_ensemble.fit(Xtrain, Ytrain)


VotingRegressor(estimators=[('randomforest',
                             RandomForestRegressor(max_depth=1000,
                                                   n_estimators=2000, n_jobs=-1,
                                                   random_state=42)),
                            ('gradientboosing',
                             GradientBoostingRegressor(n_estimators=50,
                                                       random_state=42)),
                            ('xgb',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          e...
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None, max_bin=None,
                                          max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=None,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None, n_estimators=50,
                                          n_jobs=None, num_parallel_tree=None,
                                          random_state=None, ...))],
                weights=[0.5, 0.1, 0.4])

In [ ]:
ensemble_prediction = soft_ensemble.predict(Xtest)

In [ ]:

ensemble_mean_abs_err = mean_absolute_error(Ytest, ensemble_prediction)
ensemble_mean_abs_err

0.003581736036105373

Comparing the models

In [ ]:

print("RandomForest Model: ", mean_abs_err_results)
print("ADA Model: ", mae)
print("GradientBoosting Model: ", gb_mean_abs_err)
print("XGBBoost Model: ", xg_mean_abs_err)
print("SoftEnsemble Model: ", ensemble_mean_abs_err)

RandomForest Model:  0.00038591476091476447
ADA Model:  0.0002598752598752599
GradientBoosting Model:  0.03281081265922311
XGBBoost Model:  0.00027643667685019
SoftEnsemble Model:  0.003581736036105373


From observation, the AdaBoostRegressor Model is the more accurate one so it's the preference to fine tune

#Fine tuninig

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)]
max_depth = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
min_samples_split = [2, 5, 10, 15, 20]
param_grid = {'n_estimators':n_estimators, 'max_depth':max_depth, 'min_samples_split':min_samples_split}

In [ ]:
cv=KFold(n_splits=3)

In [ ]:
#random_search = GridSearchCV(rf_model,param_grid=param_grid,cv=cv,scoring="neg_mean_absolute_error")

In [ ]:
n_estimators = [1000, 2000, 4000]
learning_rate = [0.1, 0.2, 0.3]
param_grid = {'n_estimators':n_estimators,'learning_rate':learning_rate}

In [ ]:
random_search = GridSearchCV(adaboost_regressor,param_grid=param_grid,cv=cv,scoring="neg_mean_absolute_error", n_jobs=3)



In [ ]:
random_search.fit(X, Y)

GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
             estimator=AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=1000),
                                         n_estimators=1000, random_state=42),
             n_jobs=3,
             param_grid={'learning_rate': [0.1, 0.2, 0.3],
                         'n_estimators': [1000, 2000, 4000]},
             scoring='neg_mean_absolute_error')

In [ ]:
print('best estimator: ', random_search.best_estimator_)
print('best param: ', random_search.best_params_)


best estimator:  AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=1000),
                  learning_rate=0.1, n_estimators=1000, random_state=42)
best param:  {'learning_rate': 0.1, 'n_estimators': 1000}


In [ ]:
base_model_2 = DecisionTreeRegressor(max_depth=1000)
model_2 = AdaBoostRegressor(base_model_2, n_estimators=2000, learning_rate=0.2, random_state=42)
model_2.fit(Xtrain, Ytrain)

model_prediction_2 = model_2.predict(Xtest)

In [ ]:
mean_abs_err_results_2 = mean_absolute_error(Ytest, model_prediction_2)
mean_abs_err_results_2

0.0002598752598752599

#Testing

An attempt to fix a bug

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import pickle



if 'overall' in legacy_dataset.columns:
    Y_train = legacy_dataset['overall']
    X_train = legacy_dataset.drop(columns='overall')
else:
    raise ValueError("'overall' column not found in the legacy_dataset DataFrame")

print("Columns in X_train before training:")
print(X_train.columns)


numerical_columns = X_train.select_dtypes(include=['number']).columns.tolist()




Columns in X_train before training:
Index(['player_id', 'player_url', 'fifa_version', 'fifa_update',
       'fifa_update_date', 'short_name', 'long_name', 'player_positions',
       'potential', 'value_eur',
       ...
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk',
       'player_face_url'],
      dtype='object', length=109)


In [ ]:
imputer = SimpleImputer(strategy='mean')
X_train_imputed = X_train.copy()
X_train_imputed[numerical_columns] = imputer.fit_transform(X_train[numerical_columns])



In [ ]:

sc = StandardScaler()
X_train_scaled = X_train_imputed.copy()
X_train_scaled[numerical_columns] = sc.fit_transform(X_train_imputed[numerical_columns])

In [ ]:
# Train the model
model = LinearRegression()
model.fit(X_train_scaled[numerical_columns], Y_train)


with open('model_2.pkl', 'wb') as f:
    pickle.dump(model, f)


with open('trained_feature_names.pkl', 'wb') as f:
    pickle.dump(numerical_columns, f)

print("Model and feature names saved successfully.")

Model and feature names saved successfully.


In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import pickle


with open('model_2.pkl', 'rb') as f:
    model_2 = pickle.load(f)

with open('trained_feature_names.pkl', 'rb') as f:
    trained_feature_names = pickle.load(f)


if 'overall' in players_dataset.columns:
    Y_new = players_dataset['overall']
    X_new = players_dataset.drop(columns='overall')
else:
    raise ValueError("'overall' column not found in the new dataset DataFrame")


In [ ]:

print("Columns in the new dataset (X_new):")
print(X_new.columns.tolist())

print("\nTrained feature names:")
print(trained_feature_names)

# Identify missing columns
missing_columns = [col for col in trained_feature_names if col not in X_new.columns]
print("\nMissing columns in X_new:")
print(missing_columns)

# Identify extra columns in X_new
extra_columns = [col for col in X_new.columns if col not in trained_feature_names]
print("\nExtra columns in X_new:")
print(extra_columns)


Columns in the new dataset (X_new):
['short_name', 'long_name', 'player_positions', 'potential', 'value_eur', 'wage_eur', 'age', 'dob', 'height_cm', 'weight_kg', 'club_team_id', 'club_name', 'league_name', 'league_level', 'club_position', 'club_jersey_number', 'club_joined', 'club_contract_valid_until', 'nationality_id', 'nationality_name', 'preferred_foot', 'weak_foot', 'skill_moves', 'international_reputation', 'work_rate', 'body_type', 'real_face', 'release_clause_eur', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots', 'mentality_aggression', 'ment

In [ ]:

X_new = players_dataset.drop(columns='overall')
Y_new = players_dataset['overall']

# Filter the data again
filtered_trained_feature_names = [col for col in trained_feature_names if col in X_new.columns]

print("\nFiltered trained feature names (used for prediction):")
print(filtered_trained_feature_names)

# I want to select only the numerical data
X_new = X_new[filtered_trained_feature_names]

# Impute again
imputer = SimpleImputer(strategy='mean')
X_new_imputed = pd.DataFrame(imputer.fit_transform(X_new), columns=X_new.columns)


sc = StandardScaler()
X_new_scaled = pd.DataFrame(sc.fit_transform(X_new_imputed), columns=X_new.columns)




Filtered trained feature names (used for prediction):
['potential', 'value_eur', 'wage_eur', 'age', 'height_cm', 'weight_kg', 'league_level', 'club_team_id', 'club_jersey_number', 'nationality_id', 'weak_foot', 'skill_moves', 'international_reputation', 'release_clause_eur', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots', 'mentality_aggression', 'mentality_interceptions', 'mentality_positioning', 'mentality_vision', 'mentality_penalties', 'mentality_composure', 'defending_marking_awareness', 'defending_standing_tackle', 'defending_sliding_tackle', 

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import pickle


with open('model_2.pkl', 'rb') as f:
    model_2 = pickle.load(f)

with open('trained_feature_names.pkl', 'rb') as f:
    trained_feature_names = pickle.load(f)

# Ensure the 'overall' column is in the new dataset
if 'overall' in players_dataset.columns:
    Y_new = players_dataset['overall']
    X_new = players_dataset.drop(columns='overall')
else:
    raise ValueError("'overall' column not found in the new dataset DataFrame")


print("Columns in the new dataset (X_new):")
print(X_new.columns.tolist())

print("\nTrained feature names:")
print(trained_feature_names)


missing_columns = [col for col in trained_feature_names if col not in X_new.columns]
print("\nMissing columns in X_new:")
print(missing_columns)


for col in missing_columns:
    X_new[col] = 0  # using 0 for now


X_new = X_new[trained_feature_names]



Columns in the new dataset (X_new):
['short_name', 'long_name', 'player_positions', 'potential', 'value_eur', 'wage_eur', 'age', 'dob', 'height_cm', 'weight_kg', 'club_team_id', 'club_name', 'league_name', 'league_level', 'club_position', 'club_jersey_number', 'club_joined', 'club_contract_valid_until', 'nationality_id', 'nationality_name', 'preferred_foot', 'weak_foot', 'skill_moves', 'international_reputation', 'work_rate', 'body_type', 'real_face', 'release_clause_eur', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots', 'mentality_aggression', 'ment

In [ ]:

imputer = SimpleImputer(strategy='mean')
X_new_imputed = pd.DataFrame(imputer.fit_transform(X_new), columns=X_new.columns)


sc = StandardScaler()
X_new_scaled = pd.DataFrame(sc.fit_transform(X_new_imputed), columns=X_new.columns)


players_22_prediction = model_2.predict(X_new_scaled)


from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(Y_new, players_22_prediction)
print(f"Mean Absolute Error on the new dataset: {mae}")

print("Predictions for the new dataset:")
print(players_22_prediction)


Mean Absolute Error on the new dataset: 1.5775800279169887
Predictions for the new dataset:
[99.27174977 99.43364529 98.91579117 ... 47.42620147 48.8533843
 49.8324025 ]


#Save Model

In [ ]:
import pickle

In [ ]:
# Scaler object as file to scale user input with the necessary parameters
pickle.dump(sc, open('/content/drive/MyDrive/Colab Notebooks/new_scaler_parameters.pkl', 'wb'))

# Model object as file to predict the scaled values after user input
pickle.dump(model_2, open('/content/drive/MyDrive/Colab Notebooks/new_player_ratings_model.pkl', 'wb'))